In [ ]:
import numpy as np 
import random 
from time import sleep 


class Logic:
  # Initializes an empty board. game_logic.py
  def __init__(self):
    self.board = np.full((3,3), ' ')
    self.user = input('X or O?')
    self.set_mode()
    if self.user == 'X' or 'x':
      self.computer = 'O'
    else:
      self.computer = 'X'
      
  
  # temp print board method
  def print_board(self, board):
    print(board[0,0] + '|' + board[0,1] + '|' + board[0,2])
    print('-+-+-')
    print(board[1,0] + '|' + board[1,1] + '|' + board[1,2])
    print('-+-+-')
    print(board[2,0] + '|' + board[2,1] + '|' + board[2,2])
  
  def resetBoard(self, board):
    self.board = np.full((3,3), ' ')
    return self.board


  # Check for empty places on board
  def __possibilities(self, board):
    emptySpaces = []
        
    for i in range(len(board)):
        for j in range(len(board)):    
            if board[i][j] == ' ':
                emptySpaces.append((i, j))
    return(emptySpaces)

  def set_mode(self):
    self.mode = input('Which level do you want to play on(E or H)?')
    if self.mode != 'e' or self.mode != 'E' or self.mode != 'h' or self.mode != 'H':
      self.mode = input('Which level do you want to play on(E or H)?')
    return self.mode
  
  # Easy/Beginner Mode. Game simply selects a random place for the computer
  # Returns updated board with 1 new position filled.
  def easy_mode(self, board):
    selection = self.__possibilities(board)
    current_loc = random.choice(selection)
    board[current_loc] = self.computer
    return(board)
  
  # Hard Mode. AI performs Minimax Algorithm to determine selection for computer
  def hard_mode(self, board):
    bestScore=-1000 #lowest to start with, for comaparison to the score after a certain move
    bestMove=0
    #print('hit')
    for row in range(len(board)):
      for col in range(len(board)):
        if board[row][col] == ' ':
          board[row][col]=self.computer
          score = self.minimax(board, 0, False) #calculate the score
          board[row][col] = ' ' #set board back to what it was
          if (score > bestScore): #compare the score from a certain move to the best score.
            bestScore = score
            bestMove = row, col
    
    board[bestMove] = self.computer

    return(board)


  def minimax(self, board, depth, isMaximizing):
      if (self.evaluate(board, ['X', 'O']) == self.computer):
        return 1
      elif self.evaluate(board, ['X', 'O']) is None:
        return 0
      elif self.evaluate(board, ['X', 'O']) == self.user:
        return -1
      if isMaximizing:
        bestScore = -1000
        for row in range(len(board)):
          for col in range(len(board)):
            if board[row][col] == ' ':
              board[row][col]=self.computer
              score = self.minimax(board, 0, False)
              board[row][col]= ' '
              if (score>bestScore):
                bestScore = score
        return bestScore
      else:
        bestScore = 1000
        for row in range(len(board)):
          for col in range(len(board)):
            if board[row][col] == ' ':
              board[row][col]=self.user
              score = self.minimax(board, 0, True)
              board[row][col]= ' '
              if (score<bestScore):
                bestScore = score
        return bestScore

  # Checks whether the player has three of their marks in a horizontal row
  def row_win(self, board, player):
    for row in range(len(board)):
      win = True
      for col in range(len(board)):
        if board[row, col] != player:
            win = False
            continue
      if win == True:
        return(win)
      return(win)
    
  # Checks whether the player has three of their marks in a vertical row
  def col_win(self, board, player):
    for row in range(len(board)):
      win = True
      for col in range(len(board)):
        if board[col][row] != player:
            win = False
            continue  
    if win == True:
      return(win)
    return(win)
    
  # Checks whether the player has three of their marks in a diagonal row
  def diag_win(self, board, player):
    win = True
    y = 0
    for x in range(len(board)):
      if board[x, x] != player:
        win = False
    if win:
      return win
    win = True
    if win:
      for x in range(len(board)): # check reverse diagonal
        y = len(board) - 1 - x
        if board[x, y] != player:
          win = False
    return win
    
  # Evaluates whether there is a winner or a tie 
  def evaluate(self, board, XO):
    winner = 0
        
    for player in XO:
      if (self.row_win(board, player) or
          self.col_win(board,player) or 
          self.diag_win(board,player)):
            winner = player
                
    if np.all(board != ' ') and winner == 0:
      winner = None
    return winner


# game method
global board, winner, XO, draw

def play_game():
    XO = ['X', 'O'] 

    try: 
      while True: 
        logic = Logic()
        winner_dict = {'X':0, 'O':0}
        board, winner, counter = logic.board, 0, 0
        cur_player = XO[0] # X is first
        game_over = False
        sleep(2) 
        while game_over is False: # Individual game loop
          print(XO[XO.index(cur_player)] + "'s Turn")
          if cur_player == logic.computer:
            # perform game logic based on levl/game mode
            if logic.mode == 'E' or logic.mode == 'e':
              #board = logic.easy_mode(board, XO[XO.index(cur_player)])
              board = logic.easy_mode(board)
            elif logic.mode == 'H' or logic.mode == 'h':
              board = logic.hard_mode(board)
              sleep(2) 
            counter += 1
          else: # user's turn
            valid_move = False
            while valid_move is False:
              move = input('Row, Column')
              row, col = int(move[0]), int(move[2])
              if board[row, col] == ' ':
                board[row, col] = cur_player
                valid_move = True
                counter +=1 
              else:
                print('Chose another position.')
              


          print("Board after " + str(counter) + " move")
          logic.print_board(board)

          # Change the player after every move.
          if cur_player =='X':
            cur_player = 'O'
          else:
            cur_player = 'X'    

          sleep(2)
          winner = logic.evaluate(board, XO)
          if isinstance(winner,str) or winner is None:
            game_over = True 
            if winner is None:
              print('Draw')
            else:
              winner_dict[winner] += 1
              print('Player ' + winner + ' won.')
              print(winner_dict)
            #ask if user would like to continue
            play_again = input('Play Again?')
            if play_again == 'y' or 'Y':
              logic.set_mode()
              board, winner, counter = logic.resetBoard(board), 0, 0
              game_over = False

        return(winner)
      
    # check if user quits
    except KeyboardInterrupt:
      print('Score: ')
      print( winner_dict)
      print('Game Over')
play_game()